## Galaxy-Galaxy Lenses with Power-Law - Full Sky Area 20,000 sq.deg
Author: Paras Sharma

We generate galaxy galaxy lenses population here.

In [5]:
# Standard imports
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# Astropy imports
from astropy.io import fits
from astropy import units as u
from astropy.table import Table, vstack
from astropy.units import Quantity
from astropy import constants as const
import corner

# SLSim imports
import slsim.Pipelines as pipelines
import slsim.Sources as sources
import slsim.Deflectors as deflectors
from slsim.Lenses.lens_pop import LensPop
from slsim.Plots.lens_plots import LensingPlots
from slsim.Lenses.lens import Lens
from slsim.LOS.los_pop import LOSPop

# Lenstronomy, HierArc imports
from lenstronomy.LensModel.lens_model import LensModel
from hierarc.Sampling.ParamManager.cosmo_param import CosmoParam

from warnings import filterwarnings
filterwarnings("ignore")

In [6]:
PROJECT_DIR = "~/projects/PDSPL/psdpl-analysis"  # Change this to your local path

## Generate SLSim Catalog

In [4]:
# define a cosmology
cosmology = "FwCDM"  # Flat wCDM cosmology
# other options are: "FLCDM FwCDM", "w0waCDM", "oLCDM"
kwargs_cosmo_true = {"h0": 70, "om": 0.3, "w": -1}  # cosmological model of the forecast

# create astropy.cosmology instance of input cosmology
cosmo_param = CosmoParam(cosmology=cosmology)
cosmo_true = cosmo_param.cosmo(kwargs_cosmo_true)

# define a sky area
# sky_area_galaxies = Quantity(value=100, unit="deg2")

# load red and blue galaxy catalogs
red_galaxy_catalog = Table.read(f"../data/skypy_galaxy_catalogs/SKYPY_RED_GALAXIES_{100.0}_SQDEG.fits",
                                  format="fits")
blue_galaxy_catalog = Table.read(f"../data/skypy_galaxy_catalogs/SKYPY_BLUE_GALAXIES_{100.0}_SQDEG.fits",
                                   format="fits")


In [3]:
print("# of red galaxies:", len(red_galaxy_catalog))
print("# of blue galaxies:", len(blue_galaxy_catalog))

# of red galaxies: 8461200
# of blue galaxies: 131301420


In [4]:
%%writefile ~/projects/PDSPL/notebooks/worker.py
# This entire cell will be saved as 'worker.py'

import numpy as np
import astropy.units as u
import astropy.constants as const
from astropy.cosmology import FlatLambdaCDM
from lenstronomy.LensModel.lens_model import LensModel

cosmo_true = FlatLambdaCDM(H0=70, Om0=0.3)

# --- THE WORKER FUNCTION ---
def extract_lens_properties(args):
    """
    Worker function that takes a tuple (id, lens_object), extracts all
    properties, and returns them as a single dictionary.
    """
    lens_id, lens = args
    
    # try:
    deflector = lens.deflector
    source = lens.source(index=0)

    z_D = lens.deflector_redshift
    z_S = lens.source_redshift_list[0]
    theta_E = lens.einstein_radius[0]
    sigma_v_D = deflector.velocity_dispersion()
    e1_mass_D, e2_mass_D = deflector.mass_ellipticity
    
    half_light_radii_arcsec = deflector.angular_size_light
    
    lenstronomy_kwargs = lens.lenstronomy_kwargs()
    lens_model_lenstronomy = LensModel(lens_model_list=lenstronomy_kwargs[0]["lens_model_list"])
    lenstronomy_kwargs_lens = lenstronomy_kwargs[1]["kwargs_lens"]
    
    deflector_center = deflector.deflector_center
    grid = np.linspace(-half_light_radii_arcsec, half_light_radii_arcsec, 500)
    xs, ys = np.meshgrid(grid + deflector_center[0], grid + deflector_center[1])
    
    kappa_map = lens_model_lenstronomy.kappa(xs, ys, kwargs=lenstronomy_kwargs_lens)
    mask = np.sqrt((xs - deflector_center[0])**2 + (ys - deflector_center[1])**2) < half_light_radii_arcsec / 2
    kappa_within_half_light_radii = np.nanmean(kappa_map[mask])

    D_s = cosmo_true.angular_diameter_distance(lens.source_redshift_list[0])
    D_d = cosmo_true.angular_diameter_distance(lens.deflector_redshift)
    D_ds = cosmo_true.angular_diameter_distance_z1z2(lens.deflector_redshift, lens.source_redshift_list[0])
    
    sigma_crit = (const.c**2 / (4 * np.pi * const.G)) * (D_s / (D_d * D_ds))
    sigma_crit = sigma_crit.to(u.Msun / u.pc**2).value
    surface_density = sigma_crit * kappa_within_half_light_radii

    surface_brightness_map = deflector.surface_brightness(xs, ys, band="g")
    mask_sb = np.sqrt((xs - deflector_center[0])**2 + (ys - deflector_center[1])**2) < half_light_radii_arcsec
    mean_surface_brightness = np.nanmean(surface_brightness_map[mask_sb])

    R_e_kpc_val = (cosmo_true.kpc_proper_per_arcmin(lens.deflector_redshift) * \
                    ((half_light_radii_arcsec * u.arcsec).to(u.arcmin))).to(u.kpc).value

    ### contrast ratio of images in mag difference
    contrast_ratio_i = lens.contrast_ratio(band="i", source_index = 0)
    contrast_ratio_r = lens.contrast_ratio(band="r", source_index = 0)
    contrast_ratio_g = lens.contrast_ratio(band="g", source_index = 0)
    contrast_ratio_z = lens.contrast_ratio(band="z", source_index = 0)
    contrast_ratio_y = lens.contrast_ratio(band="y", source_index = 0)

    # make all to have len of 4
    contrast_ratio_i = np.array(list(contrast_ratio_i) + [np.nan] * (4 - len(contrast_ratio_i)))
    contrast_ratio_r = np.array(list(contrast_ratio_r) + [np.nan] * (4 - len(contrast_ratio_r)))
    contrast_ratio_g = np.array(list(contrast_ratio_g) + [np.nan] * (4 - len(contrast_ratio_g)))
    contrast_ratio_z = np.array(list(contrast_ratio_z) + [np.nan] * (4 - len(contrast_ratio_z)))
    contrast_ratio_y = np.array(list(contrast_ratio_y) + [np.nan] * (4 - len(contrast_ratio_y)))

    ## magnifications of point images and extended images
    magnification_point = lens.point_source_magnification()[0]
    # convert to array of length 4
    magnification_point = np.array(list(magnification_point) + [np.nan] * (4 - len(magnification_point)))

    magnification_extended = lens.extended_source_magnification[0] # not an array, just a number

    return {
        "lens_id": lens_id, "z_D": z_D, "z_S": z_S, "theta_E": theta_E, "sigma_v_D": sigma_v_D,
        "stellar_mass_D": deflector.stellar_mass, "mag_S_i": source.extended_source_magnitude("i"),
        "mag_S_r": source.extended_source_magnitude("r"), "mag_S_g": source.extended_source_magnitude("g"),
        "mag_S_z": source.extended_source_magnitude("z"), "mag_S_y": source.extended_source_magnitude("y"),
        "mag_D_i": deflector.magnitude("i"), "mag_D_r": deflector.magnitude("r"), "mag_D_g": deflector.magnitude("g"),
        "mag_D_z": deflector.magnitude("z"), "mag_D_y": deflector.magnitude("y"), "size_D": deflector.angular_size_light,
        "e1_mass_D": e1_mass_D, "e2_mass_D": e2_mass_D, "e_mass_D": np.sqrt(e1_mass_D**2 + e2_mass_D**2),
        "gamma_pl": deflector.halo_properties['gamma_pl'], "R_e_kpc": R_e_kpc_val, "R_e_arcsec": half_light_radii_arcsec,
        "Sigma_half_Msun/pc2": surface_density, "surf_bri_mag/arcsec2": mean_surface_brightness,
        "num_images": lens.image_number[0],
        "contrast_ratio_i": contrast_ratio_i,
        "contrast_ratio_r": contrast_ratio_r,
        "contrast_ratio_g": contrast_ratio_g,
        "contrast_ratio_z": contrast_ratio_z,
        "contrast_ratio_y": contrast_ratio_y,
        "ps_magnification": magnification_point,
        "es_magnification": magnification_extended,
    }
    # except Exception as e:
    #     print(f"Error processing lens ID {lens_id}: {e}")
    #     return None

Overwriting /home/paras/projects/PDSPL/notebooks/worker.py


In [5]:
import multiprocessing
import pandas as pd
from tqdm import tqdm
from astropy.table import Table
import astropy.units as u

# --- Import the worker function from the file we just created ---
from projects.PDSPL.notebooks.worker import extract_lens_properties

In [6]:
# define a sky area
sky_area_source = Quantity(value=100, unit="deg2")  # sky area of the source population
sky_area_deflector = Quantity(value=100, unit="deg2")  # sky area of the deflector population
sky_area_lens = Quantity(value=1000, unit="deg2")  # sky area of the lens population


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

# Initiate deflector population class.
# lens_galaxies = deflectors.AllLensGalaxies(
#     red_galaxy_list=galaxy_simulation_pipeline.red_galaxies,
#     blue_galaxy_list=galaxy_simulation_pipeline.blue_galaxies,
#     kwargs_cut=kwargs_deflector_cut,
#     kwargs_mass2light=None,
#     cosmo=cosmo_true,
#     sky_area=sky_area_deflector,
# )
lens_galaxies = deflectors.EllipticalLensGalaxies(
    galaxy_list=red_galaxy_catalog,
    kwargs_cut=kwargs_deflector_cut,
    kwargs_mass2light=None,
    cosmo=cosmo_true,
    sky_area=sky_area_deflector,
    gamma_pl={"mean": 2.078, "std_dev": 0.16}, # Ref: Auger et al. 2010 [https://ui.adsabs.harvard.edu/abs/2010ApJ...724..511A/abstract], For SLACS lenses
)

# Initiate source population class.
source_galaxies = sources.Galaxies(
    galaxy_list=blue_galaxy_catalog,
    kwargs_cut=kwargs_source_cut,
    cosmo=cosmo_true,
    sky_area=sky_area_source,
    catalog_type="skypy",
)

# make galaxy-galaxy population class using LensPop
gg_lens_pop = LensPop(
    deflector_population=lens_galaxies,
    source_population=source_galaxies,
    cosmo=cosmo_true,
    sky_area=sky_area_lens,
    los_pop=LOSPop(los_bool=False),  # no line-of-sight population, can be separately included in the forecast as 1% uncertainty on beta_E
)

# make lensplot class for extracting rgb images
gg_plot = LensingPlots(gg_lens_pop, num_pix=100, coadd_years=10)

print(f"LensPop initialized. Potential deflectors: {gg_lens_pop.deflector_number} ; Potential sources: {gg_lens_pop.source_number}");


LensPop initialized. Potential deflectors: 25251010 ; Potential sources: 474717380


In [ ]:
TIMES_TO_RUN = 50

main_progress_bar = tqdm(np.arange(0, TIMES_TO_RUN, 1), desc="Overall Progress", position=0)

for i in main_progress_bar:

    kwargs_lens_cut = {"min_image_separation": 1, "max_image_separation": 8,
                    "second_brightest_image_cut": {"i": 26,},
                    }

    # kwargs_lens_cut = {}
    selected_lenses = gg_lens_pop.draw_population(
        kwargs_lens_cuts=kwargs_lens_cut,
    )

    # Print the number of selected lenses
    print(f"       Number of selected lenses: {len(selected_lenses)}")


    # 2. RUN IN PARALLEL
    num_processes = multiprocessing.cpu_count()-5
    print(f"       Starting parallel processing with {num_processes} cores...")
    multiprocessing.set_start_method("spawn", force=True)

    with multiprocessing.Pool(processes=num_processes) as pool:
        # Use enumerate() to pass both the ID and the lens object to the worker
        results_list = list(tqdm(
            pool.imap(extract_lens_properties, enumerate(selected_lenses)),
            total=len(selected_lenses),
            desc="Calculating lens properties"
        ))

    # 3. PROCESS RESULTS
    results_list = [res for res in results_list if res is not None]

    if results_list:
        df = pd.DataFrame(results_list)
        final_table = Table.from_pandas(df)
        
        # Save the table to a FITS file
        output_path = f"{PROJECT_DIR}/data/FULL_LSST/GGL_{sky_area_lens.value}_SQDEG_NON_SIS_{i}.fits"
        final_table.write(output_path, format="fits", overwrite=True)

        print(f"       Results saved to {output_path}")
        
        print("\n       Processing complete. First 5 rows of the generated table:")
        print(final_table[:5])
    else:
        print("       No results were generated.")

The above cell was restarted many times due to kernel crashes. The final run generated 50 catalogs of 1000 sq.deg each.

## Make Single Table for 20,000 sq.deg

Note that SLSim sometimes repeats deflector galaxies when generating catalogs for large sky areas so we will effectively use a larger sky area initially and then choose unique deflectors only corresponding to 20,000 sq.deg expected number of lenses.

In [ ]:
# TIMES_TO_RUN = 50

In [ ]:
# filenames_1000deg_catalogs = [f"../data/FULL_LSST/GGL_{1000.0}_SQDEG_NON_SIS_{i}.fits" for i in range(TIMES_TO_RUN)]
# fits_tables_1000deg_catalogs = [Table.read(filename, format="fits") for filename in filenames_1000deg_catalogs]
# combined_table = vstack(fits_tables_1000deg_catalogs)
# combined_table['lens_id'] = np.arange(len(combined_table))
# # save the combined table
# combined_table.write(f"../data/FULL_LSST/GGL_{TIMES_TO_RUN*1000.0}_SQDEG_NON_SIS.fits", format="fits", overwrite=True)

In [ ]:
# num_lenses_needed = int((20000.0 / (TIMES_TO_RUN*1000.0)) * len(combined_table))
# print(f"Total lenses in combined table: {len(combined_table)}")
# print(f"Total unique deflectors in combined table: {len(np.unique(combined_table['z_D']))}")
# print(f"Lenses needed for 20,000 sq.deg: {num_lenses_needed}") # These don't include any observational cuts yet

Total lenses in combined table: 1515140
Total unique deflectors in combined table: 331004
Lenses needed for 20,000 sq.deg: 606056


In [55]:
# save 10000 deg^2 worth of lenses with non-repeated deflectors
full_50k_deg2_non_sis_table = Table.read(f"../data/FULL_LSST/GGL_{50000.0}_SQDEG_NON_SIS.fits", format="fits")
unique_deflector_indices = np.unique(full_50k_deg2_non_sis_table['z_D'], return_index=True)[1]

In [56]:
num_expected_lenses_10k = len(full_50k_deg2_non_sis_table) / 5
unique_indices_for_10k = np.random.choice(unique_deflector_indices, size=int(num_expected_lenses_10k), replace=False)

unique_10k_deg2_non_sis_table = full_50k_deg2_non_sis_table[unique_indices_for_10k]
unique_10k_deg2_non_sis_table

lens_id,z_D,z_S,theta_E,sigma_v_D,stellar_mass_D,mag_S_i,mag_S_r,mag_S_g,mag_S_z,mag_S_y,mag_D_i,mag_D_r,mag_D_g,mag_D_z,mag_D_y,size_D,e1_mass_D,e2_mass_D,e_mass_D,gamma_pl,R_e_kpc,R_e_arcsec,Sigma_half_Msun/pc2,surf_bri_mag/arcsec2,num_images,contrast_ratio_i,contrast_ratio_r,contrast_ratio_g,contrast_ratio_z,contrast_ratio_y,ps_magnification,es_magnification
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,object,object,object,object,object,object,float64
161277,0.4688260952333436,1.9336322412350715,0.6258343641537345,182.90313689173354,191992670893.0238,25.37614764694804,25.493259159647884,25.729657471185014,25.08815112499049,24.91758687578774,20.02104850490003,20.815093224177218,22.332497055002797,19.6436376137297,19.429553263805197,0.28271972109600996,-0.07029008114660973,-0.018035399116859748,0.07256701129922194,2.1283234832666182,1.666321650766298,0.2827197210960111,11225.227106616336,22.0521725720167,2,[-2.83833695 -0.1992065 nan nan],[-3.51527016 -0.8761397 nan nan],[-4.79627568 -2.15714522 nan nan],[-2.74892258 -0.10979213 nan nan],[-2.70540248 -0.06627203 nan nan],[ 2.18111153 -0.87266693 nan nan],3.047561953511793
405211,2.333113893965864,3.320652921936555,0.5702383759813455,358.9357743563626,1125327461075.4026,27.221866795132673,27.401829048095063,27.74725754288345,27.062974029069974,26.97256524403581,25.009003774736303,25.202138358890434,25.441680318549434,24.765928329311603,24.16681299923715,1.3616120416866386,-0.016547230962295906,0.027369531874497328,0.03198284114253594,1.8358836646327372,11.143638243719703,1.3616120416866384,5527.5617959833435,28.579100044958196,2,[-7.53626967 -6.41623532 nan nan],[-7.549442 -6.42940765 nan nan],[-7.44355547 -6.32352111 nan nan],[-7.45208699 -6.33205264 nan nan],[-6.94338045 -5.82334609 nan nan],[ 5.28285203 -3.53894527 nan nan],9.022270456592693
12756,0.7623519254823877,3.1334698963696925,0.9366416294421297,235.43218502450475,410413080040.2853,25.63472654698019,25.75467231886263,25.84341854329263,25.543516715973347,25.59519183745214,20.54676511203576,21.48335671624923,22.177323710700268,20.0899116051887,19.8553312665759,0.3740270969340602,-0.012481623826946536,-0.0022707401321630048,0.012686496526039613,2.336942396941782,2.761116696597867,0.3740270969340604,15768.665675852328,22.507963926904058,2,[-3.39803639 -1.55793079 nan nan],[-4.21468222 -2.37457662 nan nan],[-4.81990299 -2.97979739 nan nan],[-3.03239271 -1.19228711 nan nan],[-2.74613725 -0.90603165 nan nan],[ 2.3948269 -0.91124513 nan nan],3.3109451451827296
656625,0.15166656856401642,2.329059511055345,0.5372829885627768,144.8021815418882,73406946612.14061,21.958825096328056,22.121870181338238,22.316133733516892,21.811324263867903,21.605121525053086,17.87527113603456,18.31285549519026,19.18696629316857,17.609667857858565,17.42452132724245,0.39517104005377385,0.013394349867035882,0.34558664859105187,0.3458461222751467,2.1833425092875567,1.0426815110221483,0.39517104005377385,10829.703890862816,19.63862028810338,2,[0.72493622 3.77787458 nan nan],[0.45039694 3.50333531 nan nan],[-0.2294503 2.82348806 nan nan],[0.84303866 3.89597703 nan nan],[0.82198246 3.87492082 nan nan],[ 1.46484156 -0.70500417 nan nan],2.082486682691087
1053766,1.3036405936324245,2.4688280613203952,0.5180687262029982,239.8004801766151,431348016794.56683,27.284927097012066,27.564308429842022,27.93187881330504,27.14130483980321,26.846673426869323,23.63525540261503,24.998359972775894,25.830669607787804,23.00700302418779,22.13996666047072,0.43933861160456184,0.009406367476907507,-0.04759472871189119,0.04851533726842618,2.1846728589158655,3.6818095364531636,0.43933861160456184,8442.800443691835,26.518536318434876,2,[-6.04016217 -5.54730709 nan nan],[-7.12388541 -6.63103033 nan nan],[-7.58862466 -7.09576958 nan nan],[-5.55553205 -5.06267697 nan nan],[-4.9831271 -4.49027202 nan nan],[ 5.37049017 -9.5670229

In [57]:
# save the 10k deg^2 table
unique_10k_deg2_non_sis_table.write(f"../data/FULL_LSST/GGL_{10000.0}_SQDEG_NON_SIS_UNIQUE_DEFLECTORS.fits", format="fits", overwrite=True)